In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import os
import matplotlib
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout
#from keras.utils import multi_gpu_model
from tensorflow.keras.utils import plot_model
from keras import regularizers
from tensorflow.keras.optimizers import SGD,Adam
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin

In [ ]:
import tensorflow as tf

SEED = 1
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:

sns.set_context("paper")

color_set=['deep','muted','pastel','bright','dark','colorblind']
sns.set_palette(color_set[1])





large=12
med=9
small=6
ss=10
sss=8
ssss=6



matplotlib.rcParams['figure.figsize']=(3.2,2.8)
matplotlib.rcParams['figure.dpi'] = 600

matplotlib.rcParams['font.sans-serif'] = ['Arial']   
matplotlib.rcParams['font.family']='sans-serif'
matplotlib.rcParams['font.style']='normal'   # normal (or roman), italic or oblique
matplotlib.rcParams['font.weight']='bold'   # normal bold bolder lighter, 100, 200, 300, ..., 900
matplotlib.rcParams["axes.labelweight"] = "bold"

matplotlib.rcParams['axes.titlesize'] =sss
#matplotlib.rcParams['axes.titlepad'] =-0.5
matplotlib.rcParams['legend.fontsize'] =ss
matplotlib.rcParams['axes.labelsize'] =ss
matplotlib.rcParams['xtick.labelsize'] =ss
matplotlib.rcParams['ytick.labelsize'] = ss
matplotlib.rcParams['figure.titlesize'] =sss



matplotlib.rcParams['xtick.major.width'] = 0.6
matplotlib.rcParams['ytick.major.width'] = 0.6


matplotlib.rcParams['xtick.major.size'] = 1.3
matplotlib.rcParams['ytick.major.size'] = 1.3


matplotlib.rcParams['xtick.major.pad'] = 1.5
matplotlib.rcParams['ytick.major.pad'] = 1.5



matplotlib.rcParams['axes.linewidth'] = 0.6



matplotlib.rcParams['lines.linewidth'] = 1.0
#matplotlib.rcParams['lines.markersize'] = 1.2


scatter_s=20


ref_line_color=['#4d4d4d', 'red']
ref_line_size=0.8


#matplotlib.rcParams['xtick.direction'] = 'in'

In [ ]:
data_train = pd.read_excel('data_train.xlsx')
data_test = pd.read_excel('data_test.xlsx')
data_final_test = pd.read_excel('data_final_test.xlsx')

In [ ]:
X_train_data = data_train.iloc[:,1:6]
y_train = data_train.iloc[:,0]

X_col = X_train_data.columns

X_test_data = data_test.iloc[:,1:6]
y_test = data_test.iloc[:,0]

X_final_test = data_final_test.iloc[:,1:6]
y_final_test = data_final_test.iloc[:,0]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_data)

X_train_standard=scaler.transform(X_train_data)
X_test_standard=scaler.transform(X_test_data)

X_train_standard = pd.DataFrame(X_train_standard,columns=X_col)
X_test_standard = pd.DataFrame(X_test_standard,columns=X_col)
X_train_standard

In [ ]:
X_train_standard_np=X_train_standard.values
X_test_standard_np=X_test_standard.values
y_train_np=y_train.values
y_test_np=y_test.values
y_final_test_np = y_final_test.values

In [ ]:
class CustomKerasRegressor(BaseEstimator, RegressorMixin):
    
    def __init__(self, learning_rate=0.01, neurons=64, activation='relu', hidden_layers=1, dropout_rate=0.2, batch_size=32, epochs=100):
        self.learning_rate = learning_rate
        self.neurons = neurons
        self.activation = activation
        self.hidden_layers = hidden_layers
        self.dropout_rate = dropout_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.model = None

    
    def _create_model(self, input_dim):

        model = Sequential()

        model.add(Dense(self.neurons, activation=self.activation, input_dim=input_dim,
                        kernel_initializer=tf.keras.initializers.GlorotUniform(seed=1)))    

        for _ in range(self.hidden_layers - 1):
            model.add(Dense(self.neurons, activation=self.activation, kernel_initializer=tf.keras.initializers.GlorotUniform(seed=1)))
            if self.dropout_rate > 0:
                model.add(Dropout(self.dropout_rate, seed=1))

        #model.add(Dense(1))
        model.add(Dense(1, activation='linear', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=1)))

        model.compile(optimizer=Adam(learning_rate=self.learning_rate),loss='mse', metrics=['mae'])

        return model


    def fit(self, X, y, validation_data=None, verbose=0):
        self.model = self._create_model(X.shape[1])
        if validation_data:
            X_val, y_val = validation_data
            train_model = self.model.fit(X, y, validation_data=(X_val, y_val), batch_size=self.batch_size, epochs=self.epochs, verbose=verbose)
        else:
            train_model = self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=verbose)
        self.history = train_model.history
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        y_pred = self.predict(X)
        r2 = r2_score(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        mse = mean_squared_error(y, y_pred)
        rmse = np.sqrt(mse)
        return mae

    def evaluate(self, X, y):
        y_pred = self.predict(X)
        mae = mean_absolute_error(y, y_pred)
        mse = mean_squared_error(y, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y, y_pred)
        return {'R2': r2, 'MAE': mae,'RMSE': rmse}

    def save(self, filepath):
        if self.model is not None:
            self.model.save(filepath)
        else:
            raise ValueError


In [ ]:
import time
from datetime import datetime
from sklearn.model_selection import KFold

model = CustomKerasRegressor()


param_grid = {
    'learning_rate': [0.01, 0.02],
    'neurons': [200, 300],
    'activation': ['relu', 'tanh'],
    'hidden_layers': [1, 2],
    'dropout_rate': [0.0, 0.1],
    'batch_size': [32],
    'epochs': [500]
}



start_time = time.time()
start_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"{start_datetime}")


grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', cv=5, verbose=2, n_jobs=1)

grid_result = grid.fit(X_train_standard_np, y_train_np)


end_time = time.time()
end_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
total_time = end_time - start_time


print(f"{end_datetime}")
print(f"{total_time:.2f}") 
print(grid_result.best_params_)
print(grid_result.best_score_)

In [ ]:
best_params = grid_result.best_params_
best_params_filtered = {k: v for k, v in best_params.items() if k not in ['batch_size', 'epochs']}

In [ ]:
data = pd.read_excel('data_final.xlsx')


#data = data.astype(float)
#data.info()


X_all = data.iloc[:,1:6]
y_all = data.iloc[:,0]


X_all_np=X_all.values
y_all_np=y_all.values

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler_all = StandardScaler()
best_model_all = CustomKerasRegressor(learning_rate=best_params_filtered['learning_rate'],
                                      neurons=best_params_filtered['neurons'],
                                      activation=best_params_filtered['activation'],
                                      hidden_layers=best_params_filtered['hidden_layers'],
                                      dropout_rate=best_params_filtered['dropout_rate'],
                                      #batch_size=64,
                                      epochs=500,
                                      batch_size=best_params['batch_size'],
                                      #epochs=best_params['epochs']
                                     )


train_rmse_scores = []
train_r2_scores = []
train_mae_scores = []

test_rmse_scores = []
test_r2_scores = []
test_mae_scores = []

final_test_rmse_scores = []
final_test_r2_scores = []
final_test_mae_scores = []


n = 20

for i in range(n):
    X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(X_all, y_all, test_size=0.2, shuffle=True, random_state=i)

    
    scaler_all.fit(X_all_train)
    X_all_train_standard = scaler_all.transform(X_all_train)
    X_all_test_standard = scaler_all.transform(X_all_test)
    X_final_test_standard_all = scaler_all.transform(X_final_test)
    
    best_model_all.fit(X_all_train_standard, y_all_train)

    y_all_train_pred = best_model_all.predict(X_all_train_standard)
    train_rmse = np.sqrt(mean_squared_error(y_all_train, y_all_train_pred))
    train_rmse_scores.append(train_rmse)
    train_r2 = r2_score(y_all_train, y_all_train_pred)
    train_r2_scores.append(train_r2)
    train_mae = mean_absolute_error(y_all_train, y_all_train_pred)
    train_mae_scores.append(train_mae)

    y_all_test_pred = best_model_all.predict(X_all_test_standard)
    test_rmse = np.sqrt(mean_squared_error(y_all_test, y_all_test_pred))
    test_rmse_scores.append(test_rmse)
    test_r2 = r2_score(y_all_test, y_all_test_pred)
    test_r2_scores.append(test_r2)
    test_mae = mean_absolute_error(y_all_test, y_all_test_pred)
    test_mae_scores.append(test_mae)

    y_final_test_pred_all = best_model_all.predict(X_final_test_standard_all)
    test_rmse = np.sqrt(mean_squared_error(y_final_test_np, y_final_test_pred_all))
    final_test_rmse_scores.append(test_rmse)
    test_r2 = r2_score(y_final_test_np, y_final_test_pred_all)
    final_test_r2_scores.append(test_r2)
    test_mae = mean_absolute_error(y_final_test_np, y_final_test_pred_all)
    final_test_mae_scores.append(test_mae)



results_dict = {
    'method': ['ANN'] * (3 * n),
    'set_type': ['Train'] * n + ['Test'] * n + ['Final_Test'] * n,
    'MAE': train_mae_scores + test_mae_scores + final_test_mae_scores,
    'RMSE': train_rmse_scores + test_rmse_scores + final_test_rmse_scores,
    'R2': train_r2_scores + test_r2_scores + final_test_r2_scores
}
results_df = pd.DataFrame(results_dict)


excel_filename = 'model_evaluation_results_5fea.xlsx'
results_df.to_excel(excel_filename, index=False)

In [ ]:

print('train set')
print(f'Average MAE score: {np.mean(train_mae_scores):.4f} ± {np.std(train_mae_scores):.4f}')
print(f'Average RMSE score: {np.mean(train_rmse_scores):.4f} ± {np.std(train_rmse_scores):.4f}')
print(f'Average R2 score: {np.mean(train_r2_scores):.4f} ± {np.std(train_r2_scores):.4f}')

print('\ntest set')
print(f'Average MAE score: {np.mean(test_mae_scores):.4f} ± {np.std(test_mae_scores):.4f}')
print(f'Average RMSE score: {np.mean(test_rmse_scores):.4f} ± {np.std(test_rmse_scores):.4f}')
print(f'Average R2 score: {np.mean(test_r2_scores):.4f} ± {np.std(test_r2_scores):.4f}')

print('\nindependent test set')
print(f'Average MAE score: {np.mean(final_test_mae_scores):.4f} ± {np.std(final_test_mae_scores):.4f}')
print(f'Average RMSE score: {np.mean(final_test_rmse_scores):.4f} ± {np.std(final_test_rmse_scores):.4f}')
print(f'Average R2 score: {np.mean(final_test_r2_scores):.4f} ± {np.std(final_test_r2_scores):.4f}')

print(f'\nResults have been saved to {excel_filename}')


print('\nDataFrame preview:')
print(results_df.head())



print('best test set')
min_mae = min(test_mae_scores)
min_mae_index = test_mae_scores.index(min_mae)
print('min MAE score:', min_mae, '    ', 'random_state=', min_mae_index)

min_rmse = min(test_rmse_scores)
min_rmse_index = test_rmse_scores.index(min_rmse)
print('min RMSE score:', min_rmse, '    ', 'random_state=', min_rmse_index)

max_r2 = max(test_r2_scores)
max_r2_index = test_r2_scores.index(max_r2)
print('max R2 score:', max_r2, '    ', 'random_state=', max_r2_index)

print('best independent test set')
final_min_mae = min(final_test_mae_scores)
final_min_mae_index = final_test_mae_scores.index(final_min_mae)
print('min MAE score:', final_min_mae, '    ', 'random_state=', final_min_mae_index)

final_min_rmse = min(final_test_rmse_scores)
final_min_rmse_index = final_test_rmse_scores.index(final_min_rmse)
print('min RMSE score:', final_min_rmse, '    ', 'random_state=', final_min_rmse_index)

final_max_r2 = max(final_test_r2_scores)
final_max_r2_index = final_test_r2_scores.index(final_max_r2)
print('max R2 score:', final_max_r2, '    ', 'random_state=', final_max_r2_index)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

max_index=final_max_r2_index

X_train_best,X_test_best,y_train_best,y_test_best = train_test_split(X_all,y_all,test_size=0.2,shuffle=True,random_state=max_index) 


scaler_best = StandardScaler()
scaler_best.fit(X_train_best)
X_train_best_standard = scaler_best.transform(X_train_best)
X_test_best_standard = scaler_best.transform(X_test_best)
X_final_test_standard_best = scaler_best.transform(X_final_test)


best_model_best = CustomKerasRegressor(learning_rate=best_params_filtered['learning_rate'],
                                      neurons=best_params_filtered['neurons'],
                                      activation=best_params_filtered['activation'],
                                      hidden_layers=best_params_filtered['hidden_layers'],
                                      dropout_rate=best_params_filtered['dropout_rate'],
                                      #batch_size=64,
                                      epochs=500,
                                      batch_size=best_params['batch_size'],
                                      #epochs=best_params['epochs']
                                     )
best_model_best.fit(X_train_best_standard, y_train_best)
y_train_best_pred=best_model_best.predict(X_train_best_standard)

x_best_1,y_best_1=pd.Series(y_train_best,name='y_train'),pd.Series(y_train_best_pred.squeeze(),name='y_train_pred')
sns.regplot(x=x_best_1,y=y_best_1)
plt.show

#rmse_log = np.sqrt(mean_squared_error(np.log(y_test),np.log(abs(y_pred))))
rmse = np.sqrt(mean_squared_error(y_train_best, y_train_best_pred))
r2 = r2_score(y_train_best, y_train_best_pred)


print(f'train Mean Absolute Error MAE: {mean_absolute_error(y_train_best,y_train_best_pred)}')
print ('train rmse scores : ',rmse)
print ('train R2 scores : ',r2)

In [ ]:

y_test_best_pred=best_model_best.predict(X_test_best_standard)


x_best_2,y_best_2=pd.Series(y_test_best,name='y_test'),pd.Series(y_test_best_pred.squeeze(),name='y_pred')
sns.regplot(x=x_best_2,y=y_best_2)
plt.show

rmse = np.sqrt(mean_squared_error(y_test_best,y_test_best_pred))
r2 = r2_score(y_test_best,y_test_best_pred)


print(f'test Mean Absolute Error MAE: {mean_absolute_error(y_test_best,y_test_best_pred)}')
print ('test rmse scores : ',rmse)
print ('test R2 scores : ',r2)

In [ ]:

y_final_test_best_pred=best_model_best.predict(X_final_test_standard_best)


x_best_3,y_best_3=pd.Series(y_final_test_np,name='y_final_test'),pd.Series(y_final_test_best_pred.squeeze(),name='y_pred')
sns.regplot(x=x_best_3,y=y_best_3)
plt.show

rmse = np.sqrt(mean_squared_error(y_final_test_np,y_final_test_best_pred))
r2 = r2_score(y_final_test_np,y_final_test_best_pred)


print(f'test Mean Absolute Error MAE: {mean_absolute_error(y_final_test_np,y_final_test_best_pred)}')
print ('test rmse scores : ',rmse)
print ('test R2 scores : ',r2)

In [ ]:
best_model_best.save('model_best_5fea.keras')

In [ ]:
fig = plt.figure()
ax=fig.add_subplot(111)

ax.scatter(x_best_1,y_best_1,s=20,marker='o', color='w',  edgecolors='tab:green',linewidths=1.2, label = 'Train') 
ax.scatter(x_best_2,y_best_2,s=20,marker='s', color='w', edgecolors='tab:red',linewidths=1.2, label = 'Test1')
ax.scatter(x_best_3,y_best_3,s=20,marker='^', color='w', edgecolors='tab:blue',linewidths=1.2, label = 'Test2')


x_r=[-0.48, -0.21]
y_r=[-0.48, -0.21]
ax.plot(x_r,y_r,'k--')


ax.legend(loc='upper left',
          markerscale=1.2,)


ax.set_xlabel('DFT calculated (eV)',labelpad = 1.3)
ax.set_ylabel('Predicted (eV)',labelpad = 1.3)


ax.set_xlim([-0.48,-0.21])  
ax.set_ylim([-0.48,-0.21])


xmajorLocator   = MultipleLocator(0.05) 
ax.xaxis.set_major_locator(xmajorLocator)

ymajorLocator   = MultipleLocator(0.05) 
ax.yaxis.set_major_locator(ymajorLocator)


#fig.tight_layout()
fig.tight_layout(pad=0.4, w_pad=1.0, h_pad=1.0)

plt.savefig('ANN_best_5fea.pdf')
plt.show()